In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/winning_ensemble_0702"
#path_ensemble = f"{path_work}/ficheros_28032023/ensemble_tailored_0612"


/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


> Make model : 

In [2]:
dico_models, errors = TropiGAT_functions.make_ensemble_TropiGAT(path_ensemble)

In [ ]:
len(dico_models)

> Ferriol

In [3]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.1007.csv", sep = "," , header = None, index_col = 0)


In [4]:
dpo_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
K23PH08C2__cds_228_Dpo_domain,0.014787,-0.020907,-0.000637,0.062485,-0.111420,0.049894,-0.035570,0.027610,0.061589,0.031878,...,0.086178,0.086181,-0.043126,0.151168,-0.026617,0.049532,0.055483,-0.021074,-0.118191,-0.017345
K54lambda1_1_1__cds_230_Dpo_domain,-0.000109,0.025560,-0.006387,0.075376,-0.013559,0.018169,0.008427,-0.096586,0.045259,0.079440,...,-0.016448,-0.048871,0.047981,0.120588,0.072368,-0.152706,0.116491,-0.040279,0.013591,0.079716
K23PH08C2__cds_220_Dpo_domain,0.041871,0.016468,0.008620,0.071750,0.015430,0.074183,-0.025165,-0.094952,0.053195,0.032808,...,0.041965,0.013586,0.018654,0.115373,0.045196,-0.162757,0.024553,0.034368,-0.008758,0.066617
K45PH128C2__cds_232_Dpo_domain,0.015056,0.009604,0.005614,0.084654,-0.030902,0.029202,0.000621,-0.118652,0.068846,0.074969,...,-0.005421,-0.040130,0.054284,0.146513,0.046997,-0.137585,0.106764,-0.025498,0.022369,0.082952
K45PH128C2__cds_235_Dpo_domain,0.087291,0.035496,-0.004867,0.091631,-0.129568,0.078719,-0.018489,-0.000897,0.123086,0.078086,...,0.009472,0.013997,0.037375,0.164937,0.054671,0.012141,0.075321,-0.009618,-0.030154,0.005304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K24PH164C1__cds_55_Dpo_domain,0.039957,-0.007577,-0.037035,0.054355,-0.128855,0.044301,0.099503,-0.163834,0.076119,-0.003351,...,0.016038,0.009034,0.027130,0.023492,-0.033999,-0.025833,0.030128,-0.042748,-0.122199,0.124239
K42PH8__cds_43_Dpo_domain,0.015222,-0.034378,-0.049853,0.063044,-0.158600,0.051105,0.079608,-0.183104,0.016171,-0.001478,...,0.023938,-0.037645,0.006023,0.016808,-0.034763,-0.044617,0.048723,-0.071603,-0.118990,0.130576
K10PH82C1__cds_45_Dpo_domain,0.017037,-0.028448,-0.066188,0.038587,-0.124722,0.051205,0.086921,-0.141901,0.024835,-0.017928,...,0.022757,-0.000056,0.013690,0.011025,-0.067530,-0.004214,0.050276,-0.072483,-0.137275,0.142341


> Bea

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)
bea_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
A1c_00034,-0.031228,0.018990,0.011914,-0.036915,-0.085279,0.074510,0.038491,-0.092385,0.041953,0.006984,...,0.024149,-0.001948,0.086334,0.015169,-0.009941,0.070748,0.061202,-0.088183,0.029006,0.092003
A1h_00013,0.003407,0.058613,0.004211,-0.032270,-0.080894,0.003224,0.026575,-0.049651,0.068412,0.048845,...,-0.037309,-0.002876,0.103554,-0.011835,0.040593,0.037963,0.031057,-0.102158,0.077059,0.088456
S11a_00036,-0.000087,0.046707,0.027543,-0.037515,-0.073041,0.012802,0.033908,-0.055600,0.117382,0.031282,...,-0.037399,0.023287,0.144737,-0.022651,0.048932,0.046084,0.025735,-0.100239,0.049737,0.135533
A1q_00023,0.035220,-0.004737,-0.047403,0.058447,-0.127027,0.047977,0.093704,-0.168074,0.073823,-0.002389,...,0.015234,0.019885,0.038854,0.031011,-0.032295,-0.027215,0.027464,-0.042393,-0.133780,0.124112
A1q_00010,0.006108,0.036785,0.033718,-0.061564,-0.102759,0.008589,0.025191,-0.038584,0.046164,0.036677,...,-0.030083,-0.019619,0.079121,-0.012699,0.048490,0.054487,0.050371,-0.102361,0.034225,0.098482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A1e_00012,0.023357,0.039636,-0.014646,-0.014371,-0.082256,0.020024,-0.017925,-0.063363,0.068452,0.040234,...,0.016703,-0.005695,0.106709,-0.003896,0.034324,0.069450,0.058052,-0.072267,0.086228,0.071590
M5a_00170,-0.021793,0.077967,-0.017637,-0.069029,-0.085250,0.012727,0.054289,-0.040869,0.069873,0.026029,...,-0.023246,-0.027288,0.092628,-0.031385,0.033944,0.073543,0.025255,-0.100661,0.049953,0.117564
A1j_00002,0.036885,-0.005403,-0.047889,0.057499,-0.128569,0.041746,0.098268,-0.166326,0.072257,-0.004946,...,0.011941,0.016783,0.038163,0.026237,-0.031345,-0.027221,0.030271,-0.040723,-0.132024,0.123907


> Towndsend

In [6]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.1112.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)


>Others old

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_phages.esm2.embedding.csv", sep = "," , header = None)
others_embeddings = others_embeddings.drop([1281] , axis = 1)
others_embeddings.set_index([0], inplace = True)

namesother_df = pd.read_csv(f"{path_project}/Others/index_others.tsv", sep = "\t" , names = ["index_phage", "index_prot","prot_name"])
new_index = []
for _,index in enumerate(others_embeddings.index) : 
    i_phage = int(index.split("__")[0])
    i_prot = int(index.split("__")[1])
    prot = namesother_df[(namesother_df["index_phage"] == i_phage) & (namesother_df["index_prot"] == i_prot)]["prot_name"].values[0]
    new_index.append(prot)

others_embeddings.index = new_index
others_embeddings.index.name = 0

> Others 

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

***
> run the predictions 

In [7]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    try : 
        graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
        pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
        ferriol_predictions[dpo] = pred
    except Exception as e :
        print(dpo, e)

In [8]:
ferriol_predictions["K17alfa62__cds_66_Dpo_domain"]

{'KL52': 0.7733,
 'KL55': 0.7278,
 'KL34': 0.934,
 'KL81': 0.9603,
 'KL60': 0.9406,
 'KL30': 0.8581,
 'KL70': 0.9688,
 'KL127': 0.5474,
 'KL62': 1.0,
 'KL145': 0.876,
 'KL128': 0.9958}

In [ ]:
# format the results : 
ferriol_pred_formated = TropiGAT_functions.format_predictions(ferriol_predictions , sep = "__")
TropiGAT_functions.clean_print(ferriol_pred_formated)

***

In [9]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred

In [ ]:
# format the results : 
bea_pred_formated = TropiGAT_functions.format_predictions(bea_predictions , sep = "_")
TropiGAT_functions.clean_print(bea_pred_formated)

In [10]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

In [ ]:
# format the results : 
towndsend_pred_formated = TropiGAT_functions.format_predictions(towndsend_predictions , sep = "_")
TropiGAT_functions.clean_print(towndsend_pred_formated)

***

In [ ]:
others_embeddings.info()

In [ ]:
others_embeddings.index

In [ ]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

In [ ]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

***
# Write the results : 

> Others : 

In [ ]:
predictions = [other_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.results.classic_2406.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Predictions : 

In [11]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.results.classic_1007.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Parse TropiGAT and Seqbased results :

In [12]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.results.classic_1007.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])

# Seqbased_model.results.bit50.0101.tsv
# Seqbased_model.0101.results.tsv
# Seqbased_model.1001.results.tsv
seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.results.bit75.1007.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])


In [13]:
seqbased_results

,protein,predictions_seqbased
0,K18PH07C1__cds_231,No_hits
1,K23PH08C2__cds_220,No_hits
2,K23PH08C2__cds_225,No_hits
3,K23PH08C2__cds_228,No_hits
4,K45PH128C2__cds_227,No_hits
...,...,...
265,A2a_b_00022,No_associations
266,A2a_b_00036,KL102:0.737
267,A1i_00037,KL102:0.737
268,A1i_00041,KL48:0.568


In [14]:
tropigat_results

,protein,predictions_tropigat
0,K23PH08C2__cds_228_Dpo_domain,KL114:1.0 ; KL27:1.0 ; KL123:1.0 ; KL136:1.0 ;...
1,K54lambda1_1_1__cds_230_Dpo_domain,KL39:1.0 ; KL112:1.0 ; KL23:0.999 ; KL19:0.999...
2,K23PH08C2__cds_220_Dpo_domain,KL36:1.0 ; KL74:1.0 ; KL19:1.0 ; KL34:0.999 ; ...
3,K45PH128C2__cds_232_Dpo_domain,KL39:1.0 ; KL112:1.0 ; KL19:0.999 ; KL74:0.998...
4,K45PH128C2__cds_235_Dpo_domain,KL114:1.0 ; KL27:1.0 ; KL112:1.0 ; KL123:0.999...
...,...,...
264,NBNDMPCG_00163,KL13:0.958 ; KL2:0.954 ; KL57:0.94 ; KL60:0.92...
265,NJHLHPIG_00061,KL46:0.99 ; KL128:0.975 ; KL18:0.971 ; KL52:0....
266,HIIECEMK_00054,KL60:0.995 ; KL18:0.991 ; KL23:0.943 ; KL14:0....
267,PP187_gp237,KL110:0.955 ; KL114:0.931 ; KL70:0.897 ; KL9:0...


In [15]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

,protein_x,predictions_tropigat,protein_id,protein_y,predictions_seqbased,phage
139,A1a_00014,KL45:0.878 ; KL2:0.741 ; KL74:0.57 ; KL48:0.52...,A1a_00014,A1a_00014,KL151:0.599,A1a
154,A1a_00002,KL123:0.977 ; KL14:0.911 ; KL13:0.9 ; KL39:0.8...,A1a_00002,A1a_00002,KL102:0.737,A1a
140,A1b_00048,KL128:0.919 ; KL70:0.885 ; KL52:0.862 ; KL46:0...,A1b_00048,A1b_00048,No_associations,A1b
132,A1b_00036,KL7:0.983 ; KL13:0.975 ; KL123:0.971 ; KL14:0....,A1b_00036,A1b_00036,KL102:0.737,A1b
104,A1c_00034,KL52:0.998 ; KL62:0.993 ; KL128:0.981 ; KL43:0...,A1c_00034,A1c_00034,No_hits,A1c
...,...,...,...,...,...,...
162,S13a_00036,KL18:0.959 ; KL27:0.938 ; KL60:0.87 ; KL123:0....,S13a_00036,S13a_00036,No_associations,S13a
124,S13b_00058,KL47:0.988 ; KL14:0.921 ; KL23:0.752 ; KL64:0....,S13b_00058,S13b_00058,KL63:0.641,S13b
156,S13c_00055,KL27:0.958 ; KL38:0.955 ; KL60:0.92 ; KL57:0.9...,S13c_00055,S13c_00055,No_associations,S13c
155,S13d_00057,KL14:0.999 ; KL52:0.973 ; KL38:0.954 ; KL27:0....,S13d_00057,S13d_00057,KL14:0.951,S13d


In [16]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]



In [17]:
final_df.to_csv(f"{path_project}/PPT_results.classic_1007.bit75.tsv", sep = "\t", header = True, index = False)

In [18]:
final_df

,phage,protein_id,predictions_seqbased,predictions_tropigat
139,A1a,A1a_00014,KL151:0.599,KL45:0.878 ; KL2:0.741 ; KL74:0.57 ; KL48:0.52...
154,A1a,A1a_00002,KL102:0.737,KL123:0.977 ; KL14:0.911 ; KL13:0.9 ; KL39:0.8...
140,A1b,A1b_00048,No_associations,KL128:0.919 ; KL70:0.885 ; KL52:0.862 ; KL46:0...
132,A1b,A1b_00036,KL102:0.737,KL7:0.983 ; KL13:0.975 ; KL123:0.971 ; KL14:0....
104,A1c,A1c_00034,No_hits,KL52:0.998 ; KL62:0.993 ; KL128:0.981 ; KL43:0...
...,...,...,...,...
162,S13a,S13a_00036,No_associations,KL18:0.959 ; KL27:0.938 ; KL60:0.87 ; KL123:0....
124,S13b,S13b_00058,KL63:0.641,KL47:0.988 ; KL14:0.921 ; KL23:0.752 ; KL64:0....
156,S13c,S13c_00055,No_associations,KL27:0.958 ; KL38:0.955 ; KL60:0.92 ; KL57:0.9...
155,S13d,S13d_00057,KL14:0.951,KL14:0.999 ; KL52:0.973 ; KL38:0.954 ; KL27:0....
